# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 87057034,
 'calculationPrice': 'tops',
 'change': 0.313,
 'changePercent': 0.00197,
 'close': 0,
 'closeSource': 'filacfio',
 'closeTime': None,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': None,
 'delayedPriceTime': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPrice': None,
 'extendedPriceTime': None,
 'high': 0,
 'highSource': None,
 'highTime': None,
 'iexAskPrice': 172.05,
 'iexAskSize': 113,
 'iexBidPrice': 162.17,
 'iexBidSize': 102,
 'iexClose': 160.493,
 'iexCloseTime': 1701250564171,
 'iexLastUpdated': 1716903832115,
 'iexMarketPercent': 0.014654388091229674,
 'iexOpen': 166.18,
 'iexOpenTime': 1714962775935,
 'iexRealtimePrice': 165.264,
 'iexRealtimeSize': 87,
 'iexVolume': 1527319,
 'lastTradeTime': 1728539670836,
 'latestPrice': 166.245,
 'latestSource': 'IEX real time price',
 'latestTime': '3:17:34 PM',
 'latestUpdate': 1708616564384,
 'latestVolume': None,
 'low': 0,
 'lowSource': None,
 'lowTime': None

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [4]:
pe_ratio = data['peRatio']
pe_ratio

27.15

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        if symbol == 'VIAC':
            final_dataframe = final_dataframe.append(
                pd.Series(
                    [symbol, 0, 0, 0,], index=my_columns
                ),
                ignore_index=True,
            )
        elif symbol == 'WLTW':
            final_dataframe = final_dataframe.append(
                pd.Series(
                    [symbol, 0, 0, 0,], index=my_columns
                ),
                ignore_index=True,
            )
        else:
            final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['quote']['peRatio'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,134.790,33.64,N/A
1,AAL,13.790,-4.47,N/A
2,AAP,203.181,22.1,N/A
3,AAPL,167.000,26.7,N/A
4,ABBV,150.060,24.14,N/A
...,...,...,...,...
500,YUM,121.640,23.4,N/A
501,ZBH,122.000,62.73,N/A
502,ZBRA,394.500,25.35,N/A
503,ZION,66.360,6.34,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [7]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)

final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,EBAY,55.020,2.64,N/A
1,F,16.560,3.79,N/A
2,BIO,545.520,3.99,N/A
3,NRG,37.700,4.35,N/A
4,COF,130.260,4.9,N/A
5,SYF,36.210,5.08,N/A
6,MPC,79.224,5.11,N/A
7,PRU,107.080,5.4,N/A
8,AIG,58.600,5.43,N/A
9,GS,333.100,5.5,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [9]:
portfolio_input()

Enter the value of your portfolio:1000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [10]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])

final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,EBAY,55.020,2.64,363
1,F,16.560,3.79,1207
2,BIO,545.520,3.99,36
3,NRG,37.700,4.35,530
4,COF,130.260,4.9,153
5,SYF,36.210,5.08,552
6,MPC,79.224,5.11,252
7,PRU,107.080,5.4,186
8,AIG,58.600,5.43,341
9,GS,333.100,5.5,60


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [11]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [12]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(","):
        if symbol == "VIAC":
            enterprise_value = 0
            ebitda = 0
            gross_profit = 0
            rv_dataframe = rv_dataframe.append(
                pd.Series(
                    [symbol, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], index=rv_columns
                ),
                ignore_index=True,
            )
        elif symbol == "WLTW":
            enterprise_value = 0
            ebitda = 0
            gross_profit = 0
            rv_dataframe = rv_dataframe.append(
                pd.Series(
                    [symbol, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], index=rv_columns
                ),
                ignore_index=True,
            )
        else:
            enterprise_value = data[symbol]["advanced-stats"]["enterpriseValue"]
            ebitda = data[symbol]["advanced-stats"]["EBITDA"]
            gross_profit = data[symbol]["advanced-stats"]["grossProfit"]

            try:
                ev_to_ebitda = enterprise_value / ebitda
            except TypeError:
                ev_to_ebitda = np.NaN

            try:
                ev_to_gross_profit = enterprise_value / gross_profit
            except TypeError:
                ev_to_gross_profit = np.NaN
                
            rv_dataframe = rv_dataframe.append(
                pd.Series(
                    [
                        symbol,
                        data[symbol]["quote"]["latestPrice"],
                        "N/A",
                        data[symbol]["quote"]["peRatio"],
                        "N/A",
                        data[symbol]["advanced-stats"]["priceToBook"],
                        "N/A",
                        data[symbol]["advanced-stats"]["priceToSales"],
                        "N/A",
                        ev_to_ebitda,
                        "N/A",
                        ev_to_gross_profit,
                        "N/A",
                        "N/A",
                    ],
                    index=rv_columns,
                ),
                ignore_index=True,
            )

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,131.70,N/A,33.85,N/A,7.7,N/A,6.32,N/A,23.698612,N/A,12.211164,N/A,N/A
1,AAL,14.06,N/A,-4.66,N/A,-1.14,N/A,0.2848,N/A,-10.822409,N/A,1.138186,N/A,N/A
2,AAP,208.97,N/A,21.84,N/A,4.06,N/A,1.12,N/A,11.364041,N/A,2.516849,N/A,N/A
3,AAPL,163.78,N/A,27.02,N/A,36.16,N/A,6.89,N/A,20.613008,N/A,16.915717,N/A,N/A
4,ABBV,155.13,N/A,24.13,N/A,17.28,N/A,4.8,N/A,11.745635,N/A,8.521408,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,117.76,N/A,23.29,N/A,-4.04,N/A,5.12,N/A,18.812415,N/A,8.959767,N/A,N/A
501,ZBH,120.84,N/A,64.25,N/A,1.94,N/A,3.23,N/A,19.008183,N/A,5.672467,N/A,N/A
502,ZBRA,400.45,N/A,25.63,N/A,7,N/A,3.6,N/A,18.352770,N/A,7.732934,N/A,N/A
503,ZION,63.78,N/A,6.34,N/A,1.37,N/A,2.79,N/A,4.785432,N/A,2.461456,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [13]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
28,ALXN,187.700,N/A,60.24,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,289.100,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,329.337,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,12.000,N/A,10.04,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,26.817,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,51.320,N/A,14.85,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
186,FLIR,60.140,N/A,32.13,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,39.590,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
193,FRT,121.330,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,2674.230,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [14]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [15]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [16]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(rv_dataframe[metric])

#Print the entire DataFrame    
rv_dataframe

0      0.734653
1      0.039604
2      0.477228
3      0.611881
4      0.540594
         ...   
500    0.506931
501    0.938614
502    0.592079
503    0.091089
504    0.845545
Name: PE Percentile, Length: 505, dtype: object
0       0.79802
1      0.057426
2       0.59604
3      0.972277
4      0.928713
         ...   
500    0.053465
501    0.269307
502    0.771287
503    0.162376
504    0.940594
Name: PB Percentile, Length: 505, dtype: object
0      0.813861
1      0.013861
2      0.157426
3      0.835644
4      0.717822
         ...   
500    0.742574
501    0.510891
502    0.558416
503    0.444554
504    0.936634
Name: PS Percentile, Length: 505, dtype: object
0      0.819802
1      0.049505
2      0.382178
3      0.754455
4       0.40198
         ...   
500    0.687129
501     0.70099
502    0.677228
503     0.10099
504    0.920792
Name: EV/EBITDA Percentile, Length: 505, dtype: object
0      0.815842
1      0.043564
2      0.134653
3      0.910891
4      0.633663
         ...   
5

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,131.70,N/A,33.85,0.734653,7.70,0.79802,6.3200,0.813861,23.698612,0.819802,12.211164,0.815842,N/A
1,AAL,14.06,N/A,-4.66,0.039604,-1.14,0.057426,0.2848,0.013861,-10.822409,0.049505,1.138186,0.043564,N/A
2,AAP,208.97,N/A,21.84,0.477228,4.06,0.59604,1.1200,0.157426,11.364041,0.382178,2.516849,0.134653,N/A
3,AAPL,163.78,N/A,27.02,0.611881,36.16,0.972277,6.8900,0.835644,20.613008,0.754455,16.915717,0.910891,N/A
4,ABBV,155.13,N/A,24.13,0.540594,17.28,0.928713,4.8000,0.717822,11.745635,0.40198,8.521408,0.633663,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,117.76,N/A,23.29,0.506931,-4.04,0.053465,5.1200,0.742574,18.812415,0.687129,8.959767,0.665347,N/A
501,ZBH,120.84,N/A,64.25,0.938614,1.94,0.269307,3.2300,0.510891,19.008183,0.70099,5.672467,0.394059,N/A
502,ZBRA,400.45,N/A,25.63,0.592079,7.00,0.771287,3.6000,0.558416,18.352770,0.677228,7.732934,0.536634,N/A
503,ZION,63.78,N/A,6.34,0.091089,1.37,0.162376,2.7900,0.444554,4.785432,0.10099,2.461456,0.128713,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [17]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,131.70,N/A,33.85,0.734653,7.70,0.79802,6.3200,0.813861,23.698612,0.819802,12.211164,0.815842,0.796436
1,AAL,14.06,N/A,-4.66,0.039604,-1.14,0.057426,0.2848,0.013861,-10.822409,0.049505,1.138186,0.043564,0.040792
2,AAP,208.97,N/A,21.84,0.477228,4.06,0.59604,1.1200,0.157426,11.364041,0.382178,2.516849,0.134653,0.349505
3,AAPL,163.78,N/A,27.02,0.611881,36.16,0.972277,6.8900,0.835644,20.613008,0.754455,16.915717,0.910891,0.81703
4,ABBV,155.13,N/A,24.13,0.540594,17.28,0.928713,4.8000,0.717822,11.745635,0.40198,8.521408,0.633663,0.644554
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,117.76,N/A,23.29,0.506931,-4.04,0.053465,5.1200,0.742574,18.812415,0.687129,8.959767,0.665347,0.531089
501,ZBH,120.84,N/A,64.25,0.938614,1.94,0.269307,3.2300,0.510891,19.008183,0.70099,5.672467,0.394059,0.562772
502,ZBRA,400.45,N/A,25.63,0.592079,7.00,0.771287,3.6000,0.558416,18.352770,0.677228,7.732934,0.536634,0.627129
503,ZION,63.78,N/A,6.34,0.091089,1.37,0.162376,2.7900,0.444554,4.785432,0.10099,2.461456,0.128713,0.185545


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [18]:
rv_dataframe.sort_values(by = 'RV Score', inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,VIAC,0.000,0,0.00,0.052475,0.0000,0.060396,0.0000,0.00297,0.000000,0.056436,0.000000,0.00297,0.03505
1,WLTW,0.000,0,0.00,0.052475,0.0000,0.060396,0.0000,0.00297,0.000000,0.056436,0.000000,0.00297,0.03505
2,AAL,14.060,N/A,-4.66,0.039604,-1.1400,0.057426,0.2848,0.013861,-10.822409,0.049505,1.138186,0.043564,0.040792
3,PRU,105.450,N/A,5.50,0.072277,0.6444,0.073267,0.5453,0.055446,3.585163,0.075248,0.528958,0.009901,0.057228
4,UNM,28.020,N/A,7.05,0.109901,0.4883,0.063366,0.4507,0.041584,3.268057,0.067327,0.448187,0.005941,0.057624
5,AIG,57.100,N/A,5.20,0.067327,0.7158,0.075248,0.8780,0.106931,3.389855,0.071287,0.876927,0.027723,0.069703
6,GM,41.920,N/A,6.10,0.087129,1.0117,0.09703,0.4685,0.045545,1.980147,0.059406,1.740614,0.081188,0.074059
7,LNC,62.480,N/A,8.33,0.120792,0.4953,0.065347,0.5250,0.053465,5.613403,0.132673,0.503038,0.007921,0.07604
8,MET,64.160,N/A,9.05,0.140594,0.7835,0.077228,0.7328,0.089109,4.881846,0.10495,0.742635,0.015842,0.085545
9,UAL,34.400,N/A,-5.61,0.037624,2.0500,0.290099,0.4208,0.035644,-13.344639,0.045545,1.015799,0.033663,0.088515


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [19]:
portfolio_input()

Enter the value of your portfolio:1000000


In [24]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(2, len(rv_dataframe['Ticker'])):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
    
rv_dataframe

D:\Programy\Anaconda\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,VIAC,0.000,0,0.00,0.052475,0.0000,0.060396,0.0000,0.00297,0.000000,0.056436,0.000000,0.00297,0.03505
1,WLTW,0.000,0,0.00,0.052475,0.0000,0.060396,0.0000,0.00297,0.000000,0.056436,0.000000,0.00297,0.03505
2,AAL,14.060,1422,-4.66,0.039604,-1.1400,0.057426,0.2848,0.013861,-10.822409,0.049505,1.138186,0.043564,0.040792
3,PRU,105.450,189,5.50,0.072277,0.6444,0.073267,0.5453,0.055446,3.585163,0.075248,0.528958,0.009901,0.057228
4,UNM,28.020,713,7.05,0.109901,0.4883,0.063366,0.4507,0.041584,3.268057,0.067327,0.448187,0.005941,0.057624
5,AIG,57.100,350,5.20,0.067327,0.7158,0.075248,0.8780,0.106931,3.389855,0.071287,0.876927,0.027723,0.069703
6,GM,41.920,477,6.10,0.087129,1.0117,0.09703,0.4685,0.045545,1.980147,0.059406,1.740614,0.081188,0.074059
7,LNC,62.480,320,8.33,0.120792,0.4953,0.065347,0.5250,0.053465,5.613403,0.132673,0.503038,0.007921,0.07604
8,MET,64.160,311,9.05,0.140594,0.7835,0.077228,0.7328,0.089109,4.881846,0.10495,0.742635,0.015842,0.085545
9,UAL,34.400,581,-5.61,0.037624,2.0500,0.290099,0.4208,0.035644,-13.344639,0.045545,1.015799,0.033663,0.088515


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [25]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [26]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [27]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [28]:
writer.save()